In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split #切分訓練和測試
from sklearn.model_selection import StratifiedKFold #交叉驗證
from sklearn.model_selection import cross_val_score #交叉驗證
from sklearn.metrics import accuracy_score #準確率
from sklearn.ensemble import RandomForestClassifier #隨機森林

import warnings
warnings.filterwarnings('ignore')

In [5]:
df1 = pd.read_csv("train.csv")
df2 = pd.read_csv("test.csv")
df = pd.concat([df1,df2])

In [20]:
df.head()

,Age,Fare,Family,Survived,Pclass,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,1,0.0,3,0,1,0,0,1
1,38.0,71.2833,1,1.0,1,1,0,1,0,0
2,26.0,7.9250,0,1.0,3,1,0,0,0,1
3,35.0,53.1000,1,1.0,1,1,0,0,0,1
4,35.0,8.0500,0,0.0,3,0,1,0,0,1


In [7]:
df.info()  ## 檢查遺失值

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


## 資料清理與特徵工程

In [8]:
# 補 Age , Embarked 和 Fare 遺失值 
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Embarked'] = df['Embarked'].fillna('S')
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())

In [9]:
df['Family'] = df['Parch'] + df['SibSp'] # 兄弟姊妹 + 父母小孩 = 家人

In [10]:
df = df[['Age','Embarked','Fare','Family','Sex','Survived','Pclass']]   #Pclass 乘客等級

In [11]:
df.head()

,Age,Embarked,Fare,Family,Sex,Survived,Pclass
0,22.0,S,7.2500,1,male,0.0,3
1,38.0,C,71.2833,1,female,1.0,1
2,26.0,S,7.9250,0,female,1.0,3
3,35.0,S,53.1000,1,female,1.0,1
4,35.0,S,8.0500,0,male,0.0,3


In [12]:
# 將沒有大小高度關系的類別行資料欄位 作 one hot encoding 
df = pd.get_dummies(df,columns = ['Sex'])
df = pd.get_dummies(df,columns = ['Embarked'])
df.head(5)

,Age,Fare,Family,Survived,Pclass,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,1,0.0,3,0,1,0,0,1
1,38.0,71.2833,1,1.0,1,1,0,1,0,0
2,26.0,7.9250,0,1.0,3,1,0,0,0,1
3,35.0,53.1000,1,1.0,1,1,0,0,0,1
4,35.0,8.0500,0,0.0,3,0,1,0,0,1


In [13]:
# 將沒有預測目標的train資料 拿出來
df_train = df.loc[pd.notnull(df.Survived)]
df_test = df.loc[pd.isnull(df.Survived)]
df_train

,Age,Fare,Family,Survived,Pclass,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.000000,7.2500,1,0.0,3,0,1,0,0,1
1,38.000000,71.2833,1,1.0,1,1,0,1,0,0
2,26.000000,7.9250,0,1.0,3,1,0,0,0,1
3,35.000000,53.1000,1,1.0,1,1,0,0,0,1
4,35.000000,8.0500,0,0.0,3,0,1,0,0,1
5,29.881138,8.4583,0,0.0,3,0,1,0,1,0
6,54.000000,51.8625,0,0.0,1,0,1,0,0,1
7,2.000000,21.0750,4,0.0,3,0,1,0,0,1
8,27.000000,11.1333,2,1.0,3,1,0,0,0,1
9,14.000000,30.0708,1,1.0,2,1,0,1,0,0


In [14]:
# 選定特徵及目標
x_test = df_test.drop(['Survived'],axis=1)
x = df_train.drop(['Survived'],axis=1)
y = df_train['Survived']

In [15]:
# 建立訓練與驗證資料
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=33)

## 訓練模型

In [17]:
#建立模型 隨機森林 參數調整
random_forest = RandomForestClassifier(n_estimators = 1000,    #調整訓練速度
                                       max_depth=None,         #控制樹的深度
                                       max_leaf_nodes=None,    #葉子數限制
                                       min_samples_leaf=1,     #葉子上最小樣本數
                                       min_samples_split=2,    #節點上的最小樣本數
                                      )
random_forest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
# 利用訓練資料進行交叉驗證
kfold = StratifiedKFold(n_splits=10,random_state=0)
cross_val_score(random_forest,x_train,y_train,scoring = "accuracy",cv=kfold).mean()

0.797593896713615

In [19]:
# 利用驗證資料驗證模型準確率
all_pred_label = random_forest.predict(x_val)
accuracy_score(y_val,all_pred_label)

0.8100558659217877

## 預測

In [21]:
# 預測測試資料
y_test = random_forest.predict(x_test)

In [47]:
# 將預測結果整理 成 kaggle格式

df2['Survived'] = list(y_test)
df3 = df2[['PassengerId','Survived']]
df3=df3.astype('int64')
df3.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0


In [48]:
df3.to_csv('鐵達尼號生存預測.csv',index=False)
